In [2]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
# import cudf
# import dask.dataframe as dd
import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', None, '')
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)

tensorflow_version: 1.14.0
torch_version: 1.3.1+cu100
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
# 载入模型
model_hour = '2020010310'
model_hour = '2020010316'
model_root = '/search/odin/publicData/CloudS/baili/new_rank/data/tuwen_hour_sgsapp_v1/models/16'
model_hour = '2020011013'
abtest = 8
model_root = f'/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v1/models/{abtest}'
FLAGS(sys.argv)
melt.init(tf.Graph())
sess = melt.get_session()
model_path = os.path.realpath(f'{model_root}/{model_hour}')
print(model_path)
predictor = melt.Predictor(sess=sess, model_dir=model_path)
assert predictor.graph.get_collection('index_feed')

2020-01-11 21:05:00 0:00:00 os.environ['CUDA_VISIBLE_DEVICES'] -1
2020-01-11 21:05:00 0:00:00 os.environ['CUDA_VISIBLE_DEVICES'] -1
2020-01-11 21:05:00 0:00:00 Tf dataset and Tf model train in Graph mode, support Horovod, keras False
2020-01-11 21:05:00 0:00:00 seed 70293
2020-01-11 21:05:00 0:00:00 log_level: 20
2020-01-11 21:05:00 0:00:00 valid_interval_epochs: 0.0
2020-01-11 21:05:00 0:00:00 buffer_size:0
2020-01-11 21:05:00 0:00:00 batch_size: 32 eval_batch_size: 32 batch_size_per_gpu: 32 num_gpus: 1 horovod: False
2020-01-11 21:05:00 0:00:00 model_dir /tmp/melt log_dir /tmp/melt


/search/odin/publicData/CloudS/chenghuige/rank/data/tuwen_hour_sgsapp_v1/models/8/2020011013
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /search/odin/publicData/CloudS/chenghuige/rank/data/tuwen_hour_sgsapp_v1/models/8/2020011013/model.ckpt-2020011013-1.00-7803


2020-01-11 21:05:59 0:00:58 Restoring using: 58.5492000579834


In [4]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [5]:
marks = ['tuwen', 'video']
head = 'mid,docid,dur,pred,abid,ori_lr_score,lr_score,show_time,position,video_time,impression_time, \
        interests,rea,pred_click,pred_dur,product_data,distribution'
# mid = 'e5e9b96d57c21de54eecf437e2b4f4c3f1c7de66b950'
hour = '2020011015'

In [6]:
abtest = 16
def deal(df):
  df.hour = df.hour.astype(str)
  df = df.groupby(['abtest', 'hour'], as_index=False).last()
  df = df.sort_values(by=['hour'])
  df = df.drop_duplicates()
  df.hour = pd.to_datetime(df.hour, format='%Y%m%d%H')
  return df

# 遍历infos路径获取scores信息
def gen_df(mark, start_time=None, end_time=None):
  #root = f'/search/odin/baili/video_data/data/{mark}_hour_sgsapp_v1/infos/16'  
  root = f'/search/odin/publicData/CloudS/rank/infos/{mark}/{abtest}'
  dfs = Manager().dict()
  files = glob.glob(f'{root}/*/scores')
  ps = min(len(files), cpu_count())
#   with pymp.Parallel(ps) as p:
#     for i in tqdm(p.range(len(files))):
  for i in tqdm(range(len(files))):
    file = files[i]
    hour_ = os.path.basename(os.path.dirname(file))
#     print(hour)
    if not hour_.startswith(hour):
      continue
    if not gezi.non_empty(file):
      continue
    df = pd.read_csv(file, sep='\t', header=None)
    df.columns = head.split(',')[:len(df.columns)]
  #       df = df[df.mid==mid]
    if len(df):
      df = df.assign(hour=hour_)
      dfs[hour] = df
  print('num_hours_match:', len(dfs))
  df = pd.concat(dfs.values())
  df.name = mark
  return df

In [9]:
# df_tuwen = gen_df('tuwen')
df_video = gen_df('video')
df_video[df_video.dur > 600][['dur',  'video_time']]

100%|██████████| 198/198 [00:03<00:00, 52.91it/s]


num_hours_match: 1


,dur,video_time
107,632,633
1542,758,771
2005,652,652
4687,603,603
5315,624,624
...,...,...
918950,621,621
919414,678,678
919420,831,831
919421,663,663


In [8]:
df = df_tuwen
df = df[df.ori_lr_score != -1.]
df = df[df.product_data == 'sgsapp']
# df = df[df.abid == abtest]
# df[df.mid == '0002864821039187368']

NameError: name 'df_tuwen' is not defined

In [ ]:
tfrecord_root = '/search/odin/publicData/CloudS/baili/rank/sgsapp/data/tuwen_hour_sgsapp_v1/tfrecords/'
tfrecord_dir = os.path.realpath(f'{tfrecord_root}/{hour}')
files = gezi.list_files(tfrecord_dir)

In [ ]:
from projects.feed.rank.src.tfrecord_dataset import Dataset
from projects.feed.rank.src import util
num_records_file = os.path.join(tfrecord_dir, "num_records.txt")
print(num_records_file)
total = melt.get_num_records(files) if not os.path.exists(num_records_file) else gezi.read_int_from(num_records_file)
print('total', total, file=sys.stderr)
batch_size = 512
l = []
with sess.graph.as_default():
  # if not infer very slow just use cpu to do it.. so we can train more models parallel
  dataset = Dataset('valid')
  iter = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
  op = iter.get_next()

  print('---batch_size', dataset.batch_size, FLAGS.batch_size, melt.batch_size(), file=sys.stderr)  

  num_steps = -int(-total // batch_size)
  print('----num_steps', num_steps, file=sys.stderr) 
  try:
    for _ in tqdm(range(num_steps), total=num_steps):
      x, y = sess.run(op)
      mids = np.asarray([m.decode() for m in x['mid']])
      docids = np.asarray([m.decode() for m in x['docid']])
      x['mid'] = mids
      x['docid'] = docids
      l += [x]
  except Exception:
    print('abc')
    pass

In [ ]:
import copy
l2 = copy.deepcopy(l)
# mid = '0005860595047198898'
# docid = '20102e0l3KDdu7'
mid = 'fffc868062043258335'
docid = 'open_doc_prod16543897'
for i in tqdm(range(len(l2))):
  filter_flag = l2[i]['mid'] == mid
  filter_flag *= l2[i]['docid'] == docid
  for key in l2[i]:
    try:
      l2[i][key] = l2[i][key][filter_flag]
    except Exception:
      pass
l2 = [x for x in l2 if len(x['mid'])]

In [ ]:
def predict(x):
  feed_dict = {
                predictor.graph.get_collection('index_feed')[0]: x['index'],
                predictor.graph.get_collection('value_feed')[0]: x['value'],
                predictor.graph.get_collection('field_feed')[0]: x['field'],
                predictor.graph.get_collection('uid_feed')[0]: x['uid'],
                predictor.graph.get_collection('did_feed')[0]: x['did'],
                predictor.graph.get_collection('doc_idx_feed')[0]: x['history'],
              } 
  feed_dict.update({
                  predictor.graph.get_collection('time_interval_feed')[0]: x['time_interval'].reshape(-1, 1),
                  predictor.graph.get_collection('time_weekday_feed')[0]: x['time_weekday'].reshape(-1, 1),
                  predictor.graph.get_collection('timespan_interval_feed')[0]: x['timespan_interval'].reshape(-1, 1),             
#                   predictor.graph.get_collection('product_feed')[0]: product_.reshape(-1, 1)
        })
#   return predictor.predict(['pred', 'pred_click', 'pred_dur'], feed_dict)
  return predictor.predict('pred', feed_dict)

In [ ]:
len(l2)

In [ ]:
predict(l2[0])

In [ ]:
x = copy.deepcopy(l2[0])
# x['value'][0] = np.ones_like(x['value'][0])
predict(x)
x['history']

In [ ]:
x['value'][0]

In [ ]:
def mask(x, fields):
  if isinstance(fields, str):
    fields = set(map(int, fields.split(',')))
  else:
    fields = set(fields)
  for i in range(len(x['field'])):
    flag = np.asarray([float(x !=0 and x not in fields) for x in x['field'][i]])
    x['value'][i] *= flag

In [ ]:
field_names = open('/home/gezi/mine/pikachu/projects/feed/rank/src/conf/tuwen/fields.txt').readlines()
field_names = [f'{i+1}:{x.strip()}' for i, x in enumerate(field_names)]
field_names.insert(0, '0:None')
# field_names

In [ ]:
res = []
for x in l2:
  pred = predict(x)
  res.append([x['docid'][0], x['impression_time'][0], x['position'][0], pred[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['docid', 'impression_time', 'position', 'pred']
df.sort_values('pred', ascending=False)

In [ ]:
# res = []
# l2 = copy.deepcopy(l)
# for x in l2:
#   mask(x, [47,53,89,84,55])
#   pred = predict(x)
#   res.append([x['docid'][0], x['impression_time'][0], x['position'][0], pred[0][0][0]])
# df = pd.DataFrame.from_dict(res)
# df.columns=['docid', 'impression_time', 'position', 'pred']
# df.sort_values('pred', ascending=False)

In [ ]:
res = []
for i in range(91):
  x = copy.deepcopy(l2[0])
  mask(x, [i])
  res.append([field_names[i], predict(x)[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['field', 'pred']
df.sort_values('pred')

In [ ]:
df[df.pred > 0.358][df.pred < 0.359]